# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')
item_features = pd.read_csv('../raw_data/product.csv')
user_features = pd.read_csv('../raw_data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

D:\Git\Recommendation-Systems\src\utils.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5000


In [4]:
recommender = MainRecommender(data_train_lvl_1)

In [5]:
recommender.get_als_recommendations(2375, N=200)

[844179,
 871756,
 863655,
 828370,
 864312,
 845208,
 843441,
 824738,
 851277,
 831728,
 843114,
 844916,
 870547,
 872062,
 843347,
 819765,
 854261,
 854852,
 819982,
 828278,
 844165,
 843343,
 853221,
 864335,
 819112,
 870428,
 867215,
 822161,
 820122,
 855747,
 858102,
 854270,
 872060,
 870372,
 825218,
 851016,
 863208,
 856284,
 846823,
 869029,
 847982,
 864279,
 863699,
 828106,
 823990,
 871611,
 855914,
 858373,
 835476,
 839130,
 855279,
 825343,
 863447,
 866937,
 832785,
 821930,
 843306,
 871094,
 819534,
 830304,
 827683,
 850066,
 859075,
 854842,
 851507,
 842704,
 842455,
 866528,
 819518,
 847962,
 860903,
 837495,
 871208,
 833310,
 845109,
 866211,
 853529,
 866878,
 865456,
 837167,
 869047,
 865357,
 837751,
 825358,
 825029,
 868656,
 855488,
 826835,
 859154,
 862732,
 867366,
 843111,
 841309,
 866828,
 824759,
 820133,
 819656,
 839818,
 834303,
 826067,
 867667,
 842558,
 841899,
 836445,
 845914,
 872791,
 868909,
 850139,
 865948,
 825749,
 870195,
 

In [6]:
recommender.get_own_recommendations(2375, N=200)

[859075,
 844165,
 844179,
 854852,
 845208,
 862349,
 866211,
 847066,
 835300,
 847962,
 825541,
 866871,
 857006,
 850102,
 840640,
 857503,
 823704,
 865178,
 863447,
 871756,
 825994,
 861445,
 854852,
 862349,
 844179,
 866211,
 859075,
 844165,
 845208,
 833025,
 847982,
 824005,
 871756,
 865178,
 832678,
 834117,
 868764,
 870547,
 860975,
 856772,
 857503,
 835098,
 861445,
 825541,
 854405,
 823704,
 870515,
 839419,
 863447,
 819978,
 865456,
 872137,
 821083,
 866871,
 859010,
 864774,
 863802,
 838186,
 852856,
 845307,
 828867,
 845078,
 852486,
 867188,
 849202,
 831628,
 860299,
 865528,
 858302,
 844818,
 845193,
 823990,
 834103,
 861279,
 827683,
 857006,
 819765,
 865174,
 819927,
 848071,
 871611,
 850102,
 856252,
 828106,
 845705,
 823721,
 837270,
 830887,
 865705,
 848029,
 866755,
 862732,
 827180,
 822785,
 826385,
 843756,
 842140,
 825994,
 825343,
 869577,
 871061,
 846823,
 859427,
 848319,
 871440,
 819255,
 854754,
 827656,
 840601,
 856827,
 825365,
 

In [7]:
recommender.get_similar_items_recommendation(2375, N=200)

[833025,
 863447,
 833849,
 854852,
 866755,
 844179,
 824465,
 825218,
 838186,
 863447,
 870547,
 871611,
 845208,
 828106,
 823990,
 854852,
 819656,
 845208,
 844179,
 828106,
 854852,
 843346,
 855747,
 864774,
 855325,
 824005,
 821976,
 842140,
 864774,
 870547,
 841266,
 838733,
 834129,
 844175,
 870515,
 847631,
 844175,
 871061,
 848029,
 855325,
 864774,
 854852,
 862349,
 844179,
 866211,
 859075,
 844165,
 845208,
 833025,
 847982,
 824005,
 871756,
 865178,
 832678,
 834117,
 868764,
 870547,
 860975,
 856772,
 857503,
 835098,
 861445,
 825541,
 854405,
 823704,
 870515,
 839419,
 863447,
 819978,
 865456,
 872137,
 821083,
 866871,
 859010,
 864774,
 863802,
 838186,
 852856,
 845307,
 828867,
 845078,
 852486,
 867188,
 849202,
 831628,
 860299,
 865528,
 858302,
 844818,
 845193,
 823990,
 834103,
 861279,
 827683,
 857006,
 819765,
 865174,
 819927,
 848071,
 871611,
 850102,
 856252,
 828106,
 845705,
 823721,
 837270,
 830887,
 865705,
 848029,
 866755,
 862732,
 

In [8]:
recommender.get_similar_users_recommendation(2375, N=200)

[865456,
 872060,
 819112,
 861279,
 866540,
 864312,
 825289,
 841309,
 833598,
 870195,
 869577,
 839243,
 847982,
 867188,
 870547,
 871611,
 855279,
 842558,
 867188,
 819978,
 853354,
 868683,
 860501,
 859237,
 834117,
 844278,
 833025,
 831557,
 869177,
 835530,
 845078,
 827667,
 825751,
 865357,
 869910,
 855138,
 872508,
 869835,
 870073,
 868764,
 866292,
 829001,
 829291,
 827194,
 832785,
 819063,
 857773,
 865569,
 852543,
 848029,
 835530,
 842140,
 863449,
 863655,
 864893,
 869047,
 832678,
 828935,
 828106,
 827975,
 827261,
 824915,
 823842,
 823576,
 823348,
 821976,
 821930,
 820122,
 819978,
 831763,
 834103,
 850139,
 849870,
 846833,
 846823,
 845914,
 844818,
 844441,
 843441,
 843030,
 840501,
 838186,
 837366,
 837304,
 837167,
 835431,
 835243,
 835004,
 851676,
 851824,
 852360,
 852856,
 873130,
 869029,
 867931,
 867065,
 859191,
 856060,
 856046,
 855914,
 855488,
 854976,
 854496,
 853221,
 824005,
 828993,
 832678,
 842368,
 846429,
 846652,
 848970,
 

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [9]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [10]:
# Для чистоты эксперимента удалим новых юзеров

result_lvl_1 = result_lvl_1[result_lvl_1['user_id'].isin(data_train_lvl_1['user_id'])]

result_lvl_1['als_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_1['similar_items_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=200))
result_lvl_1['similar_users_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=200))
result_lvl_1['own_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
result_lvl_1['top_rec'] = [recommender.overall_top_purchases[:200] for x in np.arange(len(result_lvl_1))]
result_lvl_1.head(2)

,user_id,actual,als_rec,similar_items_rec,similar_users_rec,own_rec,top_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 852662, 845379, 838738, 824180, 85485...","[833940, 833988, 843346, 854405, 867188, 84417...","[854754, 839766, 846823, 856345, 859589, 83244...","[856942, 854852, 862349, 844179, 847982, 86545...","[854852, 862349, 844179, 866211, 859075, 84416..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[860361, 821787, 833436, 32916, 825976, 827631...","[867622, 840255, 852486, 170879, 852856, 86179...","[851101, 852600, 821787, 862010, 827631, 85679...","[854852, 844179, 862349, 865705, 820301, 83359...","[854852, 862349, 844179, 866211, 859075, 84416..."


In [11]:
recall_als = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec'], row['actual'], k=200), axis=1).mean()
recall_similar_items = result_lvl_1.apply(lambda row: recall_at_k(row['similar_items_rec'],
                                                                  row['actual'], k=200), axis=1).mean()
recall_similar_users = result_lvl_1.apply(lambda row: recall_at_k(row['similar_users_rec'],
                                                                  row['actual'], k=200), axis=1).mean()
recall_own = result_lvl_1.apply(lambda row: recall_at_k(row['own_rec'], row['actual'], k=200), axis=1).mean()
recall_top = result_lvl_1.apply(lambda row: recall_at_k(row['top_rec'], row['actual'], k=200), axis=1).mean()\

print('''
Recall@200:

als_rec = {0},
similar_items_rec = {1},
similar_users_rec = {2},
own_rec = {3},
top_rec = {4}
'''.format(recall_als, recall_similar_items, recall_similar_users, 
           recall_own, recall_top))


Recall@200:

als_rec = 0.03709955580472624,
similar_items_rec = 0.04274994572434383,
similar_users_rec = 0.016770202337439807,
own_rec = 0.04709253716628431,
top_rec = 0.04378959727277339



Видим что own recommendations и top-popular дают лучший recall.  
Далее посмотрим на различные k для ALS.

In [12]:
result_lvl_1['als_rec20'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=20))
result_lvl_1['als_rec50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=50))
result_lvl_1['als_rec100'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=100))
result_lvl_1['als_rec500'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=500))
result_lvl_1['als_rec300'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=300))


result_lvl_1.head(2)

,user_id,actual,als_rec,similar_items_rec,similar_users_rec,own_rec,top_rec,als_rec20,als_rec50,als_rec100,als_rec500,als_rec300
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 852662, 845379, 838738, 824180, 85485...","[833940, 833988, 843346, 854405, 867188, 84417...","[854754, 839766, 846823, 856345, 859589, 83244...","[856942, 854852, 862349, 844179, 847982, 86545...","[854852, 862349, 844179, 866211, 859075, 84416...","[856942, 852662, 845379, 838738, 824180, 85485...","[856942, 852662, 845379, 838738, 824180, 85485...","[856942, 852662, 845379, 838738, 824180, 85485...","[856942, 852662, 845379, 838738, 824180, 85485...","[856942, 852662, 845379, 838738, 824180, 85485..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[860361, 821787, 833436, 32916, 825976, 827631...","[867622, 840255, 852486, 170879, 852856, 86179...","[851101, 852600, 821787, 862010, 827631, 85679...","[854852, 844179, 862349, 865705, 820301, 83359...","[854852, 862349, 844179, 866211, 859075, 84416...","[860361, 821787, 833436, 32916, 825976, 827631...","[860361, 821787, 833436, 32916, 825976, 827631...","[860361, 821787, 833436, 32916, 825976, 827631...","[860361, 821787, 833436, 32916, 825976, 827631...","[860361, 821787, 833436, 32916, 825976, 827631..."


In [13]:
recall_als200 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec'], row['actual'], k=200), axis=1).mean()
recall_als20 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec20'], row['actual'], k=20), axis=1).mean()
recall_als50 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec50'], row['actual'], k=50), axis=1).mean()
recall_als100 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec100'], row['actual'], k=100), axis=1).mean()
recall_als300 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec300'], row['actual'], k=300), axis=1).mean()
recall_als500 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec500'], row['actual'], k=500), axis=1).mean()

print('''
ALS Recall@200 = {0},
ALS Recall@20 = {1},
ALS Recall@50 = {2},
ALS Recall@100 = {3},
ALS Recall@300 = {4},
ALS Recall@500 = {5}
'''.format(recall_als200, recall_als20, recall_als50, 
           recall_als100, recall_als300, recall_als500))


ALS Recall@200 = 0.03709955580472624,
ALS Recall@20 = 0.014145009784508604,
ALS Recall@50 = 0.020681497716865595,
ALS Recall@100 = 0.027474259660428296,
ALS Recall@300 = 0.044915415256377905,
ALS Recall@500 = 0.055223086118621034



Разумным значением k будет являться 200-300, далее идет естественный затухающий рост, но брать больше нет смысла, т.к. recall увеличивается незначительно, а эффективность\скорость модели второго уровня будет падать от увеличения кол-ва товаров.

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [14]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
users_lvl_2.head(3)

,user_id,candidates
0,2070,"[859075, 844179, 829955, 827667, 830192, 82089..."
1,2021,"[844179, 836445, 844165, 871756, 845208, 83557..."
2,1753,"[862981, 863762, 853998, 831559, 820486, 86876..."


In [15]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['drop'] = 1  # фиктивная пересенная

users_lvl_2.head()

,user_id,item_id,drop
0,2070,859075,1
0,2070,844179,1
0,2070,829955,1
0,2070,827667,1
0,2070,830192,1


In [16]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('drop', axis=1, inplace=True)
targets_lvl_2.head()

,user_id,item_id,target
0,2070,859075,1.0
1,2070,844179,0.0
2,2070,829955,0.0
3,2070,827667,0.0
4,2070,830192,0.0


Добавляем фичи. Начнем с фичей пользователей - средний чек и средняя сумма покупки в категории. Добавляем фичи товаров - Цена и средняя цена товара в категории

In [17]:
user_purchases = data_train_lvl_2.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
user_mean_bill = user_purchases.groupby('user_id')['sales_value'].mean().reset_index()
user_mean_bill.rename(columns={'sales_value': 'mean_bill'}, inplace=True)

data = data_train_lvl_2.merge(item_features, on='item_id', how='left')

user_mean_dep_purchase = data.groupby(['user_id', 'sub_commodity_desc'], sort=False)['sales_value'].mean().reset_index()
user_mean_dep_purchase.rename(columns={'sales_value': 'mean_dep_purchase'}, inplace=True)

In [18]:
targets_lvl_2 = targets_lvl_2.merge(user_mean_bill, on='user_id', how='left')
targets_lvl_2.head()

,user_id,item_id,target,mean_bill
0,2070,859075,1.0,14.355581
1,2070,844179,0.0,14.355581
2,2070,829955,0.0,14.355581
3,2070,827667,0.0,14.355581
4,2070,830192,0.0,14.355581


In [19]:
data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
items_price = data.groupby(['item_id'])['price'].mean().reset_index()
dep_mean_item_price = data.groupby(['sub_commodity_desc'])['price'].mean().reset_index()
dep_mean_item_price.rename(columns={'price': 'mean_dep_price'}, inplace=True)

In [20]:
items_deps = data[['item_id', 'sub_commodity_desc']].copy().drop_duplicates()
targets_lvl_2 = targets_lvl_2.merge(items_deps, on='item_id', how='left')

In [21]:
targets_lvl_2 = targets_lvl_2.merge(items_price, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(dep_mean_item_price, on='sub_commodity_desc', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_mean_dep_purchase, on=['user_id', 'sub_commodity_desc'], how='left')
targets_lvl_2.drop('sub_commodity_desc', axis=1, inplace=True)
targets_lvl_2= targets_lvl_2.fillna(0)
targets_lvl_2.head()

,user_id,item_id,target,mean_bill,price,mean_dep_price,mean_dep_purchase
0,2070,859075,1.0,14.355581,1.575195,1.917284,2.673333
1,2070,844179,0.0,14.355581,3.737419,3.892002,10.450000
2,2070,829955,0.0,14.355581,2.130000,1.871538,0.000000
3,2070,827667,0.0,14.355581,2.455556,1.611895,1.190000
4,2070,830192,0.0,14.355581,1.990000,1.611895,1.190000


Подаем данные в модель

In [22]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [23]:
%%time

lgb = LGBMClassifier(objective='binary', max_depth=7)
lgb.fit(X_train, y_train)

C:\Users\horne\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\horne\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 1.67 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=7,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [24]:
train_preds = lgb.predict_proba(X_train)

train_preds = pd.DataFrame(train_preds, columns=['0', '1'])
train_preds.head()

,0,1
0,0.732820,0.267180
1,0.507286,0.492714
2,0.999996,0.000004
3,0.978708,0.021292
4,0.955075,0.044925


In [25]:
targets_lvl_2['target_pred'] = train_preds['1']
targets_lvl_2 = targets_lvl_2.sort_values(['user_id', 'target_pred'], ascending=[False, False])
targets_lvl_2


,user_id,item_id,target,mean_bill,price,mean_dep_price,mean_dep_purchase,target_pred
275519,2500,871061,0.0,41.700000,6.565758,7.066471,6.99,1.629126e-01
275395,2500,839035,0.0,41.700000,3.990000,3.562941,4.98,1.607840e-01
275503,2500,824663,0.0,41.700000,5.790000,5.763390,5.79,1.419002e-01
275497,2500,856455,1.0,41.700000,2.495000,3.308350,2.50,1.358863e-01
275511,2500,839419,0.0,41.700000,7.000714,7.066471,6.99,1.335852e-01
...,...,...,...,...,...,...,...,...
176220,1,821292,0.0,48.825714,2.290000,2.498178,0.00,3.708436e-14
176053,1,819424,0.0,48.825714,0.000000,0.000000,0.00,3.032164e-14
176133,1,819312,0.0,48.825714,0.000000,0.000000,0.00,3.032164e-14
176167,1,820429,0.0,48.825714,0.000000,0.000000,0.00,3.032164e-14


Результат предсказания модели второго уровня сохраняем в preds_lgbm

In [26]:
preds_lgbm = targets_lvl_2[['user_id', 'item_id']].copy().drop_duplicates()
preds_lgbm = preds_lgbm.groupby('user_id')['item_id'].apply(list).reset_index()
preds_lgbm

,user_id,item_id
0,1,"[867188, 837569, 872137, 848761, 862349, 82814..."
1,2,"[866211, 859075, 843306, 836916, 838136, 87157..."
2,4,"[826099, 827667, 849870, 866025, 828353, 83106..."
3,6,"[845208, 834303, 857006, 827424, 864741, 87190..."
4,7,"[859075, 845294, 843756, 853643, 819210, 87042..."
...,...,...
2130,2496,"[865456, 844179, 824915, 859075, 867188, 82727..."
2131,2497,"[845208, 864774, 844179, 857503, 860439, 85907..."
2132,2498,"[847302, 859075, 863479, 845462, 856252, 82207..."
2133,2499,"[859075, 873023, 827683, 851676, 851468, 83088..."


Сравниваем precision@5 модели 1го уровня и двухуровневой модели на data_val_lvl_2

In [27]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.rename(columns={'item_id': 'actual'}, inplace=True)
result_lvl_2 = result_lvl_2[result_lvl_2['user_id'].isin(train_users)]

result_lvl_2['als_1'] = result_lvl_2['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_2 = result_lvl_2.merge(preds_lgbm, on='user_id', how='left')
result_lvl_2.rename(columns={'item_id': 'lgbm_2'}, inplace=True)

result_lvl_2.head(2)

,user_id,actual,als_1,lgbm_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 852662, 845379, 838738, 824180, 85485...","[867188, 837569, 872137, 848761, 862349, 82814..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[844179, 870547, 871756, 844165, 859075, 83267...",NaN


Возникла ситуация, когда некоторые юзеры есть в первой части, отсуствуют во второй и снова появляются в третьей. Для таких модель второго уровня не выдает рекомендаций, и для замеров метрики придется их удалить.

In [28]:
result_lvl_2 = result_lvl_2[result_lvl_2['lgbm_2'].notna()]
result_lvl_2.head()

,user_id,actual,als_1,lgbm_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 852662, 845379, 838738, 824180, 85485...","[867188, 837569, 872137, 848761, 862349, 82814..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[847529, 867469, 865510, 850240, 849618, 84278...","[845208, 834303, 857006, 827424, 864741, 87190..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[849032, 846634, 828219, 868539, 850962, 86621...","[859075, 845294, 843756, 853643, 819210, 87042..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[866211, 836286, 847277, 831407, 850834, 85835...","[844179, 859075, 842886, 832746, 864988, 83555..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[863762, 820486, 872146, 842249, 826090, 86118...","[844179, 820582, 858001, 827393, 857902, 86127..."


In [29]:
precision5_als_1 = result_lvl_2.apply(lambda row: precision_at_k(row['als_1'], row['actual']), axis=1).mean()
precision5_lgmb_2 = result_lvl_2.apply(lambda row: precision_at_k(row['lgbm_2'], row['actual']), axis=1).mean()


print('''
One-level Precision@5 = {0},
Two-Level Precision@5 = {1},

'''.format(precision5_als_1, precision5_lgmb_2))


One-level Precision@5 = 0.06862848134524478,
Two-Level Precision@5 = 0.09837099316868068,




У двухуровневой модели Precision@5 выше примерно на 2,5%

### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - money precision@5. Порог для уcпешной сдачи проекта money precision@5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- **2 новых товара** (юзер никогда не покупал)
- **1 дорогой товар, > 7 долларов**
- **Все товары из разных категорий** (категория - department)  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)